# Neural Networks Playground

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron #Classical Perceptron

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import Input

## Utility functions

### 🎯 Visualizing the Decision Boundary of Our MLP

The goal of this plot is to **visually inspect how our trained MLP classifier behaves** on a 2D dataset (e.g., `make_moons`).

Here's what the plot shows:

- The **background color** represents the **predicted class** by the model for every point in the 2D space.
  - Each region is colored based on the class the model assigns to it.
- The **blue contour line** is the **decision boundary**, i.e., where the model is uncertain (output ≈ 0.5).
- The **dots** are the original input points from our dataset:
  - Their **color reflects the true label**, not the model's prediction.
  - This allows us to compare the ground truth with the model's decisions.

This visualization helps us understand whether the model:
- Successfully captures the shape of the classes.
- Overfits or underfits the data.
- Struggles in specific areas of the input space.

This is a useful diagnostic tool when learning about neural networks and classification!


In [ ]:
def plot_decision_boundary(model, X, y, scaler, title="Decision Boundary"):
    from matplotlib.colors import ListedColormap
    from sklearn.metrics import pairwise_distances_argmin

    # Create the grid
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                         np.linspace(y_min, y_max, 500))

    # Display model predictions on the grid
    X_grid = np.c_[xx.ravel(), yy.ravel()]
    X_grid_scaled = scaler.transform(X_grid)
    Z = model.predict(X_grid_scaled)
    Z = Z.reshape(xx.shape)

    # Define Custom colors
    background_cmap = ListedColormap(["red", "green"])
    point_cmap = ListedColormap(["firebrick", "forestgreen"])

    # Display model predictions (chart background)
    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.2, cmap=background_cmap)

    # Display model boundary
    plt.contour(xx, yy, Z, levels=[0.5], colors='midnightblue', linewidths=2)

    # Scatter plot (assign each observation a color according to the labelled dataset)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=point_cmap, edgecolors='k', s=30)

    # Chart styles
    plt.title(title, fontsize=14)
    plt.xlabel("$x_1$", fontsize=12)
    plt.ylabel("$x_2$", fontsize=12)
    plt.xticks([])
    plt.yticks([])
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


## Dataset **Make Moons**

### Sample **Make Moons** dataset

In [ ]:
# Sample the dataset
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)

In [ ]:
# Visualize dataset
plt.figure(figsize=(6, 5))

point_cmap = ListedColormap(["firebrick", "forestgreen"]) # Set the colors

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=point_cmap, edgecolors='k', s=30)
plt.title("Make Moons dataset", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Dataset splitting and scaling

Before training a model, we divide our dataset into two parts:
- **Training set**: the data the model learns from.
- **Test set**: data the model has never seen — used to evaluate its generalization ability.

In our case, we use 80% of the data for training and 20% for testing:


In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

In [ ]:
print(f"X_train mean: {X_train.mean():.2f}")
print(f"X_train standard deviation: {X_train.std():.2f}")

In [ ]:
sns.histplot(X_train[:, 0], kde=True, label='Before Scaling')
plt.title('Distribution of Feature Before Scaling')
plt.xlabel('Feature Value')
plt.ylabel('Frequency')

# Calculate and display the mean
mean = X_train[:, 0].mean()
plt.axvline(mean, color='navy', linestyle='--', label=f'Mean: {mean:.2f}')


plt.legend()
plt.show()

Display feature distribution

### 🔍 Why we scale the data with `StandardScaler`

Neural networks are sensitive to the scale of input features.  
If one feature has values ranging from 0 to 1 and another from 1.000 to 10.000, the model might prioritize the larger one—even if it's not more important.

To prevent this, we use **standardization**:
- It transforms each feature so that it has:
  - **Mean = 0**
  - **Standard deviation = 1**

This is done using `StandardScaler` from `sklearn`:


In [ ]:
# Dataset normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(f"X_train_scaled mean: {X_train_scaled.mean():.2f}")
print(f"X_train_scaled standard deviation: {X_train_scaled.std():.2f}")

In [ ]:
# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=True)  # sharex=True for same x-axis

# Plot the distribution before scaling on the first subplot
sns.histplot(X_train[:, 0], kde=True, label='Before Scaling', ax=axes[0])
axes[0].set_title('Distribution of Feature Before Scaling')
axes[0].set_xlabel('Feature Value')
axes[0].set_ylabel('Frequency')

# Calculate and display the mean
mean_before = X_train[:, 0].mean()
axes[0].axvline(mean_before, color='navy', linestyle='-', label=f'Mean: {mean_before:.2f}')
axes[0].legend()  # Enable legend to show mean

# Plot the distribution after scaling on the second subplot
sns.histplot(X_train_scaled[:, 0], kde=True, label='After Scaling', color='orange', ax=axes[1])
axes[1].set_title('Distribution of Feature After Scaling')
axes[1].set_xlabel('Feature Value')
axes[1].set_ylabel('Frequency')
axes[1].legend()  # Enable legend for the second subplot

# Calculate and display the mean
mean_after = X_train_scaled[:, 0].mean()
axes[1].axvline(mean_before, color='navy', linestyle='--', label=f'Mean before: {mean_before:.2f}')
axes[1].axvline(mean_after, color='orange', linestyle='-', label=f'Mean after: {mean_after:.2f}')
axes[1].legend()  # Enable legend to show mean


plt.tight_layout()  # Adjust spacing between subplots
plt.show()

In [ ]:
# Visualize scaled training set
plt.figure(figsize=(6, 5))

point_cmap = ListedColormap(["firebrick", "forestgreen"])

plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], c=y_train, cmap=point_cmap, edgecolors='k', s=30)
plt.title("Make Moons Dataset (scaled)", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## Build a classifier using Neural Networks

### The Classical Perceptron

In [ ]:
# Create and train the perceptron
perceptron = Perceptron(max_iter=1000, random_state=42)
perceptron.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate the performance
y_pred = perceptron.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"Perceptron Test Accuracy: {acc:.2f}")

In [ ]:
# Plot the decision boundary
plot_decision_boundary(perceptron, X, y, scaler, title="Perceptron Decision Boundary")


### Simple Neural Network: Multi Layer Perceptron

In [ ]:
# Define the MLP model
model = models.Sequential([
    Input(shape=(2,)),
    layers.Dense(10, activation='relu'),
    layers.Dense(5, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

### Compile the MLP model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Train the MLP model

In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), verbose=0)

In [ ]:
# Evaluate performance
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test accuracy: {accuracy:.2f}")

### Plot training loss

In [ ]:
# Plot training loss
plt.plot(history.history['loss'], label='Train Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plot the decision boundary
plot_decision_boundary(model, X, y, scaler, title="Neural Network Decision Boundary")